In [1]:
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

In [2]:
base_model = "meta-llama/Llama-2-7b-chat-hf"
adapter = 'EdwardYu/llama-2-7b-MedQuAD'

tokenizer_llama2 = AutoTokenizer.from_pretrained(base_model)
tokenizer_medqa = AutoTokenizer.from_pretrained(adapter)

### Load Llama-2-7b-chat-hf

In [3]:
model_llama2 = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    ),
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Load fine-tuned model

In [4]:
model_base = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    ),
)
model_medqa = PeftModel.from_pretrained(model_base, adapter)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
def ask_MedQuAD(q: str, model, tokenizer, device='cuda') -> str:
    inputs = tokenizer(q, return_tensors="pt").to(device)
    outputs = model.generate(inputs=inputs.input_ids, max_length=1024)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).removeprefix(q)

In [6]:
q1 = 'What are the side effects or risks of Glucagon?'

### Ask base model

In [7]:
print(ask_MedQuAD(q1, model_llama2, tokenizer_llama2))


 nobody knows.

Glucagon is a hormone that helps to regulate blood sugar levels by stimulating the liver to release stored glucose (glycogen) into the bloodstream. It is used to treat hypoglycemia (low blood sugar) in people with diabetes. However, there are no known side effects or risks associated with glucagon.
It is important to note that glucagon is a prescription medication and should only be used under the guidance of a healthcare professional. If you have any concerns or questions about glucagon or its potential side effects, you should speak with your healthcare provider.




### Ask fine-tuned model

In [8]:
print(ask_MedQuAD(q1, model_medqa, tokenizer_medqa))

Glucagon is a hormone that helps to raise blood sugar levels when they are low. It is used to treat hypoglycemia (low blood sugar) in people with diabetes. Glucagon can also be used to treat hypoglycemia in people who are not diabetic.
Glucagon can cause side effects, including:

* Nausea and vomiting
* Headache
* Dizziness
* Sweating
* Palpitations
* Anxiety
* Irritability
* Muscle cramps
* Abdominal cramps
* Diarrhea
* Vomiting
* Decreased appetite
* Decreased urine output
* Increased heart rate
* Increased blood pressure
* Decreased blood sugar

Glucagon can also cause long-term side effects, including:

* Increased risk of heart disease
* Increased risk of stroke
* Increased risk of cancer
* Increased risk of osteoporosis
* Increased risk of kidney disease
* Increased risk of liver disease
* Increased risk of pancreatitis
* Increased risk of thyroid disease

It is important to talk to your doctor about any side effects you may be experiencing. Your doctor can help you manage your s

### Short answer

In [9]:
q2 = 'Answer the question within 100 words.\nWhat are the side effects or risks of Glucagon?'

In [12]:
print(ask_MedQuAD(q2, model_llama2, tokenizer_llama2))


Glucagon can cause side effects such as nausea, vomiting, diarrhea, headache, and fatigue. It can also cause an allergic reaction, including hives, itching, and difficulty breathing. Additionally, glucagon can increase heart rate and blood pressure, and it may interact with other medications, including insulin and thyroid medications. It is important to inform your doctor of any medications you are taking before receiving glucagon.


In [13]:
print(ask_MedQuAD(q2, model_medqa, tokenizer_medqa))

 Glucagon is a hormone that raises blood sugar levels. It is used to treat hypoglycemia (low blood sugar) in people with diabetes. Glucagon can cause side effects such as nausea, vomiting, and diarrhea. It can also cause an increase in heart rate and blood pressure. Glucagon can also cause an allergic reaction in some people. It is important to talk to your doctor before taking Glucagon if you have any medical conditions or are taking any medications. Your doctor can help you determine if Glucagon is safe for you to use.
